# Flask: практика, Python ФиКЛ, 2025-2026

Сегодня мы будем отрабатывать все то, что обсуждали на прошлых парах. Чтобы жить было веселее, напишем сайт для уже знакомой нам маленькой ImDB базы. 

Что хочется в итоге видеть на сайте:
- Стартовую страницу: любое содержание, можете вставить любимый мем;
- Страницу с рейтингом фильмов: выводим топ-n по рейтингу;
- Страницу с поиском фильмов: там могут быть любые фильтры;
- Страница с результатами поиска;
- Страницы с информацией о фильме и об актере.

Для тех, кто идет вперед: возможно, успеем не все, так что задания расположены в том порядке, в котором их стоит делать (а то страницы с информацией у вас повиснут в воздухе).


## Стартовая страница

Работа с шаблонами во фласке на самом деле несколько мощнее, чем мы с вами успели посмотреть. Например, вы можете вкладывать шаблоны дргу в друга. Сделаем таким образом верхнее меню для сайта.

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>{% block title %} {% endblock %}</title>
         <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@4.5.3/dist/css/bootstrap.min.css" integrity="sha384-TX8t27EcRE3e/ihU7zmQxVncDAy5uIKz4rEkgIXeMed4M0jlfIDPvg6uqKI2xXr2" crossorigin="anonymous">

    <!-- Option 2: jQuery, Popper.js, and Bootstrap JS-->
    <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js" integrity="sha384-DfXdz2htPH0lsSSs5nCTpuj/zy4C+OGpamoFVy38MVBnE+IbbVYUew+OrCXaRkfj" crossorigin="anonymous"></script>
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@4.5.3/dist/js/bootstrap.bundle.min.js" integrity="sha384-ho+j7jyWK8fNQe+A12Hb8AhRq26LrZ/JpcUGGOn+Y7RsweNrtN/tE3MoK7ZeZDyx" crossorigin="anonymous"></script>
</head>
<body>
    <!-- https://getbootstrap.com/docs/4.5/components/navbar/ -->
    <nav class="navbar navbar-expand-lg navbar-light bg-light">
      <a class="navbar-brand" href="#">База фильмов</a>
      <button class="navbar-toggler" type="button" data-toggle="collapse" data-target="#navbarNavAltMarkup" aria-controls="navbarNavAltMarkup" aria-expanded="false" aria-label="Toggle navigation">
        <span class="navbar-toggler-icon"></span>
      </button>
      <div class="collapse navbar-collapse" id="navbarNavAltMarkup">
        <div class="navbar-nav">
          <a class="nav-link" href="/rating">Рейтинг фильмов</a>
{#          <a class="nav-link" href="/people">Актеры</a>#}
          <a class="nav-link" href="/search">Поиск</a>
        </div>
      </div>
    </nav>

    <div style="margin: 50px 10%">
        {% block content %} {% endblock %}
    </div>


</body>
</html>
```

Здесь стили берутся с сайта getbootstrap.com, который предоставялет множество красиво оформленных кусочков для сайтов. Feel free to use, так сказать.

Нам для понимания работы шаблонов важна вот эта строка `<title>{% block title %} {% endblock %}</title>` и вот этот кусок:

```html
    <div style="margin: 50px 10%">
        {% block content %} {% endblock %}
    </div>
```

Именно на месте `{% block content/title %} {% endblock %}` будет располагаться все то, чем мы будем этот шаблон дополнять.

Чтобы сделать файл базовой страницы сайта, нам теперь не надо заново описывать меню и вообще весь head. Можно просто:

```html
{% extends 'base.html' %}

{% block title %} База фильмов {% endblock %}

{% block content %}
    <h1>Текст главной страницы</h1>
{% endblock %}
```

`extends` здесь сообщает, какой файл шаблона мы будем дополнять, а дальше идут блоки `content` и `title` соответсвенно.

Теперь у нас есть стартовая страница, попробуйте запустить сайт. 

В коде меню есть недочет: мы привыкли, что нажатие на кнопку с названием сайта в левом углу приводит нас обратно на стартовую страницу. Посмотрите, как оформлены другие ссылки в меню и подумайте, как наш недочет можно исправить (нужно поменять буквально один символ!).

## Страница с рейтингом

Для рейтинга нам уже требуется подключение к базе. Вспомним, как оно делается:

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy


app = Flask(__name__)


db_path = 'imdb_very_small.db'
app.config['SQLALCHEMY_DATABASE_URI'] = f'sqlite:///{db_path}'
db = SQLAlchemy(app)

Однако для работы нам нужна модель базы, с которой мы будем работать. Можно не писать всю модель сразу, а заносить в нее таблицы по мере их необходимости. Вспомним формат такой схемы с прошлой пары:

```python
class User(db.Model):
    __tablename__ = 'user'  # имя таблицы
    id = db.Column(db.Integer, primary_key=True) # имя колонки = специальный тип (тип данных, первичный ключ)
    gender = db.Column(db.Text)
    education = db.Column(db.Text)
    age = db.Column(db.Integer)


class Questions(db.Model):
    __tablename__ = 'questions'
    id = db.Column(db.Integer, primary_key=True)
    text = db.Column(db.Text)


class Answers(db.Model):
    __tablename__ = 'answers'
    id = db.Column(db.Integer, primary_key=True)
    q1 = db.Column(db.Integer)
    q2 = db.Column(db.Integer)
```

Напишите такую схему для таблиц с рейтингом и фильмами. 
Схема базы на всякий случай:

<img src="https://github.com/hse-ling-python/seminars/blob/master/databases/img/imdb_schema.png?raw=1\">

Теперь, используя `base.html` как базу, напишите шаблон для страницы с рейтитнгов, а дальше и функцию, которая будет отрисовывать страницу сайта. Не копируйте содержимое `base.html` в новый файл!

При выполнении дальнейших заданий вам может потребоваться инструкция, как переводить привычные нам SQL запросы в то, что поймет SQLAlchemy. Вот тут есть [шпаргалка](https://github.com/hse-ling-python/seminars/blob/master/flask_applications/sqlalchemy_cheatsheet.md), она нуждается в проверке, так как собиралась методом "с миру по нитке", но в среднем является полезной.